# Statistical Modeling

In [29]:
# Import Dataset and useful libraries 
import os 
import pandas as pd 
import numpy as np
os.chdir('../scripts/')
import utils as util 


data_path = "../Data/dataset.csv"
data_dict = util.read_csv_file(data_path)
df = data_dict.get('data')
print(df.columns)
df.head()


c:\Users\Administrator\Documents\Projects\Tech company\TenX\projects\AlphaCare_Insurance_ML\scripts\utils.py:5: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


Index(['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth',
       'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language',
       'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province',
       'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode',
       'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders',
       'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors',
       'VehicleIntroDate', 'CustomValueEstimate', 'AlarmImmobiliser',
       'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'WrittenOff',
       'Rebuilt', 'Converted', 'CrossBorder', 'NumberOfVehiclesInFleet',
       'SumInsured', 'TermFrequency', 'CalculatedPremiumPerTerm',
       'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section',
       'Product', 'StatutoryClass', 'StatutoryRiskType', 'TotalPremium',
       'TotalClaims'],
      dtype='object')


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


In [14]:
# Handling Missing Data
# Impute missing values for TotalPremium and TotalClaims with the mean
df['TotalPremium'].fillna(df['TotalPremium'].mean(), inplace=True)
df['TotalClaims'].fillna(df['TotalClaims'].mean(), inplace=True)

# Feature Engineering
# Create a new feature: Claim Ratio
df['ClaimRatio'] = df['TotalClaims'] / df['TotalPremium'].replace(0, np.nan)  # Avoid division by zero

# Check if categorical columns exist before encoding
categorical_columns = ['CoverType', 'CoverGroup', 'StatutoryClass', 'StatutoryRiskType']
existing_categorical_columns = [col for col in categorical_columns if col in df.columns]

# One-hot encoding for existing categorical variables
if existing_categorical_columns:
    df = pd.get_dummies(df, columns=existing_categorical_columns, drop_first=True)

# Ensure all columns are of the correct type to avoid DtypeWarning
df['TotalPremium'] = df['TotalPremium'].astype(float)
df['TotalClaims'] = df['TotalClaims'].astype(float)

# Train-Test Split
from sklearn.model_selection import train_test_split
X = df.drop(['TotalPremium', 'TotalClaims'], axis=1)  # Features
y = df[['TotalPremium', 'TotalClaims']]  # Target variables
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [30]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Handling Missing Data
# Impute missing values for TotalPremium and TotalClaims with the mean
df['TotalPremium'].fillna(df['TotalPremium'].mean(), inplace=True)
df['TotalClaims'].fillna(df['TotalClaims'].mean(), inplace=True)

# Feature Engineering
# Create a new feature: Claim Ratio
df['ClaimRatio'] = df['TotalClaims'] / df['TotalPremium'].replace(0, np.nan)  # Avoid division by zero

# Check if categorical columns exist before encoding
categorical_columns = ['CoverType', 'CoverGroup', 'StatutoryClass', 'StatutoryRiskType']
existing_categorical_columns = [col for col in categorical_columns if col in df.columns]

# One-hot encoding for existing categorical variables
if existing_categorical_columns:
    df = pd.get_dummies(df, columns=existing_categorical_columns, drop_first=True)

# Train-Test Split
X = df.drop(['TotalPremium', 'TotalClaims'], axis=1)  # Features
y = df[['TotalPremium', 'TotalClaims']]  # Target variables
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Preprocessing TransactionMonth
if 'TransactionMonth' in X_train.columns:
    X_train['TransactionMonth'] = pd.to_datetime(X_train['TransactionMonth'], errors='coerce')
    X_train['TransactionMonthNumeric'] = (X_train['TransactionMonth'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1D')
    X_train.drop(columns=['TransactionMonth'], inplace=True)

    X_test['TransactionMonth'] = pd.to_datetime(X_test['TransactionMonth'], errors='coerce')
    X_test['TransactionMonthNumeric'] = (X_test['TransactionMonth'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1D')
    X_test.drop(columns=['TransactionMonth'], inplace=True)

# Handle categorical columns
categorical_columns = ['Citizenship', 'LegalType', 'Title', 'Language', 'Bank']
X_train = pd.get_dummies(X_train, columns=categorical_columns, drop_first=True)
X_test = pd.get_dummies(X_test, columns=categorical_columns, drop_first=True)

# Check and handle duplicate column names in X_train
if X_train.columns.duplicated().any():
    print("Duplicate columns found in X_train. Resolving...")
    X_train = X_train.loc[:, ~X_train.columns.duplicated()]

# Align test set with training set columns
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Ensure TotalPremium in y_train and y_test is numeric
y_train['TotalPremium'] = pd.to_numeric(y_train['TotalPremium'], errors='coerce')
y_test['TotalPremium'] = pd.to_numeric(y_test['TotalPremium'], errors='coerce')

# Drop rows with NaN values in TotalPremium for both training and testing sets
y_train_cleaned = y_train['TotalPremium'].dropna()
X_train_cleaned = X_train.loc[y_train_cleaned.index]

# Ensure all features in X_train_cleaned are numeric
non_numeric_cols = X_train_cleaned.select_dtypes(include=['object', 'category']).columns
if not non_numeric_cols.empty:
    raise ValueError(f"Non-numeric columns still exist in X_train_cleaned: {non_numeric_cols}")

# Initialize models
linear_model = LinearRegression()
random_forest_model = RandomForestRegressor(random_state=42)

# Fit Linear Regression model
linear_model.fit(X_train_cleaned, y_train_cleaned)
y_pred_linear = linear_model.predict(X_test)

# Fit Random Forest model
random_forest_model.fit(X_train_cleaned, y_train_cleaned)
y_pred_rf = random_forest_model.predict(X_test)

# Model Evaluation
# Calculate metrics for Linear Regression
mse_linear = mean_squared_error(y_test['TotalPremium'].dropna(), y_pred_linear)
r2_linear = r2_score(y_test['TotalPremium'].dropna(), y_pred_linear)

# Calculate metrics for Random Forest
mse_rf = mean_squared_error(y_test['TotalPremium'].dropna(), y_pred_rf)
r2_rf = r2_score(y_test['TotalPremium'].dropna(), y_pred_rf)

# Print model performance comparison
print("Linear Regression - MSE: {:.2f}, R2: {:.2f}".format(mse_linear, r2_linear))
print("Random Forest - MSE: {:.2f}, R2: {:.2f}".format(mse_rf, r2_rf))

# Feature Importance Analysis for Random Forest
importances_rf = random_forest_model.feature_importances_
indices_rf = np.argsort(importances_rf)[::-1]

print("Feature ranking for Random Forest:")
for f in range(X_train_cleaned.shape[1]):
    print("%d. feature %s (%f)" % (f + 1, X_train_cleaned.columns[indices_rf[f]], importances_rf[indices_rf[f]]))


ValueError: Non-numeric columns still exist in X_train_cleaned: Index(['AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province',
       'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'VehicleType', 'make',
       'Model', 'bodytype', 'VehicleIntroDate', 'AlarmImmobiliser',
       'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'WrittenOff',
       'Rebuilt', 'Converted', 'CrossBorder', 'TermFrequency',
       'ExcessSelected', 'CoverCategory', 'Section', 'Product'],
      dtype='object')